# key for theyworkforyou API
DN8s9LBm8jMBFZihXEG2gqzx

In [1]:
APIkey = 'DN8s9LBm8jMBFZihXEG2gqzx'

In [ ]:
import requests

In [ ]:
def getQuota():
    quotaurl = 'https://www.theyworkforyou.com/api/getQuota'
    response = requests.get(quotaurl, params={'key':APIkey})
    return response.json()['quota']

In [ ]:
getQuota()

In [ ]:
import postcodes_uk

In [ ]:
postcodes_uk.validate('GL205FL')

In [ ]:
def getMP(postcode):
    postcode = postcode.upper()
    params = {'key':APIkey}
    if postcodes_uk.validate(postcode):
        params['postcode'] = postcode
    else:
        params['constituency'] = postcode
    
    url = 'https://www.theyworkforyou.com/api/getMP'
    response = requests.get(url, params=params)
    return response.json()

In [ ]:
MP = getMP('Tewkesbury')

In [ ]:
MP

In [ ]:
def getHansard(personID):
    params = {'key':APIkey}
    params['person'] = personID
    params['num'] = 512
    url = 'https://www.theyworkforyou.com/api/getHansard'
    response = requests.get(url, params=params)
    data = response.json()
    params['page'] = 1
    print('Getting %d results' % data['info']['total_results'])
    while True:
        params['page'] += 1
        response = requests.get(url, params=params)
        data0 = response.json()
        if len(data0['rows']) == 0:
            break
        data['rows'].extend(data0['rows'])
        print('%d : %d' % (len(data['rows']), data['info']['total_results']))
    return data

In [ ]:
data = getHansard(MP['person_id'])

In [ ]:
#data['rows'][-1]

In [ ]:


from bs4 import BeautifulSoup
BeautifulSoup(data['rows'][0]['body'], "html.parser").text



In [ ]:
from bs4 import BeautifulSoup
import string
extracts = [BeautifulSoup(x['body'], 'html.parser').text.lower().translate(str.maketrans('', '', string.punctuation)) for x in data['rows']]

In [ ]:
extracts[0]


In [ ]:
extracts = [e for e in extracts if len(e) > 512]

In [ ]:
100.0 * len([x for x in extracts if 'horse' in x]) / len(extracts)

In [ ]:
from wordcloud import WordCloud

In [ ]:
import matplotlib.pyplot as plt

# lower max_font_size
wc = WordCloud(max_font_size=40, background_color="black")
wc.stopwords = wc.stopwords.union({'speaker', 'secretary',
                                   'state', 'statement',
                                  'will', 'make', 'available'
                                   'made', 'ask', 'steps', 'b',
                                  'assessment', 'made', 'whether',
                                  'ensure', 'taking', 'take',
                                  'use', 'member', 'minister', 'friend', 'members', 'hon', 'mr','gentleman', 'lady', 'u', ';'})
wordcloud = wc.generate('\n'.join(extracts).lower())
plt.figure( figsize=(25,15) )
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
import nltk
nltk.download("stopwords")
nltk.download("punkt")


In [ ]:
from rake_nltk import Rake
rake_nltk_var = Rake()
keywords = []
for e in extracts:
    rake_nltk_var.extract_keywords_from_text(e)
    keywords.append(rake_nltk_var.get_ranked_phrases()[:8])

In [ ]:
# lower max_font_size
wc = WordCloud(max_font_size=40, background_color="black")
wc.stopwords = wc.stopwords.union({'speaker', 'secretary',
                                   'state', 'statement',
                                  'will', 'make', 'available'
                                   'made', 'ask', 'steps', 'b',
                                  'assessment', 'made', 'whether',
                                  'ensure', 'taking', 'take',
                                  'use', 'member', 'minister', 'friend',
                                'members', 'hon', 'mr','gentleman',
                                 'lady', 'u', ';',
                                 'people', 'many'})
wordcloud = wc.generate(' '.join([' '.join(k) for k in keywords]))
plt.figure( figsize=(25,15) )
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
rake_nltk_var.get_ranked_phrases()[:8]

In [ ]:
K = ' '.join([' '.join(k) for k in keywords])

In [ ]:
K[:128]

In [ ]:
K.count(' ')